## 3. WELCOME TO LANGCHAIN

In [2]:
from langchain_openai import ChatOpenAI # type: ignore
chat = ChatOpenAI(model_name="o4-mini")
question="How many planets are there?"
# chat_response = chat.predict(question) #this method(predict) will soon be deprecated
chat_response = chat.invoke(question)
print(chat_response)


content='If you mean “planets in our Solar System,” there are eight currently recognized planets (in order from the Sun):  \n 1. Mercury  \n 2. Venus  \n 3. Earth  \n 4. Mars  \n 5. Jupiter  \n 6. Saturn  \n 7. Uranus  \n 8. Neptune  \n\nPluto was reclassified as a “dwarf planet” by the International Astronomical Union in 2006, so it’s no longer counted among the eight.\n\nBeyond our Solar System, astronomers have confirmed over 5,500 exoplanets orbiting other stars (as of mid-2024), and thousands more candidates await confirmation. Estimates suggest our Milky Way alone may host on the order of 100–400 billion planets, and when you multiply that by the roughly two trillion galaxies in the observable universe, the total number of planets likely runs into the 10^23–10^24 range.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 533, 'prompt_tokens': 12, 'total_tokens': 545, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tok

### 3.1 Predict Messages

In [7]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italin."),
    AIMessage(content="Ciao! Mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and France? And what's your name?")
]

# res = chat.predict_messages(messages) #this method(predict_messages) will be soon deprecated.
res = chat.invoke(messages)
print(res)

content='La distanza in linea d’aria tra Città del Messico e Parigi (capitali di Messico e Francia) è di circa 9.100 chilometri.  \nMi chiamo Paolo.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 445, 'prompt_tokens': 52, 'total_tokens': 497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpSZ5ykJifXMVpgHmW83n3XXMp1S', 'finish_reason': 'stop', 'logprobs': None} id='run-0879f96f-2566-4069-a11b-f08a2aff606d-0' usage_metadata={'input_tokens': 52, 'output_tokens': 445, 'total_tokens': 497, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}}


### 3.2 Prompt Templates
* template을 디스크에 저장하고 load할 수 있음.

In [10]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore
from langchain.prompts import PromptTemplate, ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

#PromptTempalte
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Korea", country_b="France")
chat.invoke(prompt)

#ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao! Mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? And what's your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.invoke(prompt)

AIMessage(content='Η ευθύγραμμη (αεροπορική) απόσταση μεταξύ Πόλης του Μεξικού και Μπανγκόκ στην Ταϊλάνδη είναι περίπου 15\xa0900 χιλιόμετρα.  \nΤο όνομά μου είναι Βοηθός.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1040, 'prompt_tokens': 52, 'total_tokens': 1092, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpeZkfHyYLZMyPpY2cKQTLEGLPBO', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f1c67e9-7445-4c06-8b18-226395c20f59-0', usage_metadata={'input_tokens': 52, 'output_tokens': 1040, 'total_tokens': 1092, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 960}})

### 3.3 OutputParser and LCEL
* LCEL : LangChain Expression Language

In [22]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])


prompt = template.format_messages(
    max_items = 10,
    question="Give me 10 most popular colors."
)

colors = chat.invoke(prompt)
p.parse(colors.content)

['blue',
 'red',
 'green',
 'black',
 'white',
 'yellow',
 'purple',
 'orange',
 'pink',
 'brown']

In [23]:
# chain을 이용해서 위 코드를 개선할 수 있음
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5, 
    "question":"Give me the most popular pokemons."
})

['pikachu', 'charizard', 'eevee', 'mewtwo', 'bulbasaur']

### 3.4 Chaining Chains
* callbacks & streaming=True : 답변 생성이 전부 완료될 때까지 기다리지 않음. 생성되는 즉시 결과를 확인.
* callback: LLM에서 일어나는 다양한 event를 감지할 수 있음.

In [25]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

#normal recipes
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy-to-follow recipes for any type of cuising with easy-to-find ingredients."),
    ("human", "I want to cook {country} food.")
])
chef_chain = chef_prompt | chat

#vegetarian recipes
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized in making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative, just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

#mixing two chains
final_chain = {"recipe":chef_chain} | veg_chef_chain

#calling OpenAI API
final_chain.invoke({
    "country":"korean"
})

Here are three classic, easy-to-find Korean dishes you can try at home. Pick one (or all three!), gather the ingredients, and follow the step‑by‑step.  

1. Kimchi Fried Rice (Kimchi Bokkeumbap)  
Ingredients (2 servings)  
 • 2 cups cold cooked rice (preferably a day old)  
 • 1 cup well-fermented kimchi, chopped (reserve 2 Tbsp kimchi juice)  
 • ½ onion, diced  
 • 1 small carrot, julienned (optional)  
 • 2 scallions, sliced  
 • 1 Tbsp gochujang (Korean red‑pepper paste) or 1 tsp gochugaru + 1 Tbsp soy sauce  
 • 1 Tbsp soy sauce  
 • 1 tsp sugar  
 • 1 tsp sesame oil  
 • 1 Tbsp neutral oil (vegetable/canola)  
 • 2 eggs  
 • Toasted sesame seeds (for garnish)  

Steps  
1. Prep: Heat a large nonstick skillet over medium–high.  
2. Aromatics: Add neutral oil → sauté onion and carrot 2–3 min until soft.  
3. Kimchi: Add chopped kimchi + 2 Tbsp kimchi juice → cook 2 min.  
4. Seasoning: Stir in gochujang (or gochugaru+soy), soy sauce, sugar → mix.  
5. Rice: Break up cold rice with

AIMessage(content='Here’s how you can turn each of these into a vegetarian (or even vegan) dish, with minimal tweaks to the original recipes.  \n\n1. Kimchi Fried Rice  \n   • Kimchi: Most store‑bought kimchis use fish sauce or shrimp paste. Look for “vegan kimchi” at Korean markets or make your own by substituting the fish ingredients with a little crushed nori (seaweed) and a splash of soy sauce or tamari in your chili paste.  \n   • Eggs (optional): If you eat eggs, keep the sunny‑side‑up as written. For a vegan version, crumble firm tofu and sauté it in a little oil, a pinch of turmeric for color, plus salt and black salt (kala namak) for that eggy aroma. Stir it in at the very end or serve on top.  \n\n2. “Mushroom Bulgogi” (in place of Beef Bulgogi)  \n   Ingredients (2–3 servings)  \n    • 300\xa0g (10\xa0oz) king oyster mushrooms (or shiitakes + cremini), thinly sliced lengthwise into “scallops”  \n    • 3\xa0Tbsp soy sauce  \n    • 2\xa0Tbsp brown sugar or maple syrup  \n    •

## 4. MODEL IO
* <b>Modules</b>: Model I/O, Retrieval, Chains, Agents, Memory, Callbacks
* <b>Model I/O</b> : Interface with language models<br/>
   - prompts, language models, output parsers
* <b>Retrieval</b>: 외부 데이터로 접근하여 이를 모델에 제공
   - document loaders, transformers, text embedding, vector stores, retrievers
* <b>Agents</b>: AI가 독립적으로 작동할 수 있게 함.
* <b>Callbacks</b>: Log and stream intermediate steps

### 4.1 FewShotPromptTemplate
* 모델에게 예제를 주는 것과 같음 (대답 포맷 등). 답변의 형식을 구구절절 설명하는 것보다 더 빠르고 정확할 수 있음.

In [27]:
#PromptTemplate을 이렇게 만들 수도 있음 (그냥 알고 있으라고)
t = PromptTemplate(
    template = "What is the capital of {country}?",
    input_variables = ["country"]
)
#same as PromptTemplate.from_template("What is the capital of {country}?")
t.format(country="France")

'What is the capital of France?'

In [35]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({"country":"Poland"})

AI: The capital is Warsaw, the language is Polish, and the food includes pierogi, kielbasa, and bigos.

AIMessage(content='AI: The capital is Warsaw, the language is Polish, and the food includes pierogi, kielbasa, and bigos.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-400ce6e5-c9b3-44c7-97c8-f8ae7dbe8190-0')

### 4.2 FewShotChatMessagePromptTemplate

In [36]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"country":"France", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"country":"Korea", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"country":"Italy", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"country":"Japan", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai","{answer}")
])

fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert"),
    fewshot_example_prompt, #이전의 모든 example들
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke("USA")


capital is Washington D.C., 
language is English, 
food is burgers and apple pie.

AIMessage(content='capital is Washington D.C., \nlanguage is English, \nfood is burgers and apple pie.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-343e758b-b342-4748-9a4a-df05eab6528d-0')

### 4.3 LengthBasedExampleSelector
* dynamic하게 example들을 선택 -> 셀 수 없을 정도로 많은 example들이 있을 수 있음. 전부 모델한테 줄 수는 없어.


In [42]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length=30
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector, #전에는 그냥 모든 example들을 넣었음
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Netherlands")

'Human:What do you know about France?\nAI:capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup.\n\nHuman: What do you know about Netherlands?'

In [46]:
# 랜덤하게 example들을 선택
from langchain.prompts.example_selector.base import BaseExampleSelector

from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

# 나만의 example selector를 만들 수 있다는 간단한 예시
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example): #ExampleSelector 내부에 반드시 있어야 함 (없으면 에러)
        return super().add_example(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] #하나의 example만 선택해서 리턴

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples = examples
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector, #전에는 그냥 모든 example들을 넣었음
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Netherlands")

'Human:What do you know about Italy?\nAI:capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza.\n\nHuman: What do you know about Netherlands?'

### 4.4 Serialization and Composition
* serialize(직렬화): 불러오기, 저장 등
* compose: 작은 prompt templates들을 결합


In [50]:
# prompt 불러오기
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
print(prompt.format(country="China"))

prompt = load_prompt("./prompt.yaml")
print(prompt.format(country="China"))

What is the capital of China?
What is the capital of China?


In [51]:
# 여러 개의 prompt를 하나로 합치기
from langchain.prompts.pipeline import PipelinePromptTemplate

from langchain_openai import ChatOpenAI # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore
from langchain.prompts import PromptTemplate # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}.
"""
)
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human:{example_question}
    You: {example_answer}
"""
)
start=PromptTemplate.from_template(
    """
    Start now!

    Human:{question}
    You:
"""
)
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts=[
    ("intro",intro),
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat
chain.invoke(
    {
        "character":"Pirate",
        "example_question":"What is your location?",
        "example_answer":"Arrrrg! That is a scret!! Arg arrrg!",
        "question": "What is your favorite food?"
    }
)

/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/1575831411.py:52: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  full_prompt = PipelinePromptTemplate(


Arrr matey! Me favorite food be grilled fish and salted pork, aye! Nothing like a good meal to keep me strength up on the high seas. Arrrr!

AIMessage(content='Arrr matey! Me favorite food be grilled fish and salted pork, aye! Nothing like a good meal to keep me strength up on the high seas. Arrrr!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8c1d7fe1-eb46-44e4-911a-06350539baab-0')

### 4.5 Caching
* 질문, 답변을 저장. 중복되는 API 콜을 줄일 수 있음.

In [54]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

from langchain_openai import ChatOpenAI # type: ignore

# set_llm_cache(InMemoryCache()) #이제부터 모든 response가 메모리에 저장됨. 컴퓨터를 재시동하면 없어짐.
set_llm_cache(SQLiteCache("cache.db")) #sqlite 데이터베이스 생성. 여기에 response를 저장함.
set_debug(True) #로그를 보여줌. 나중에 체인 작업을 할 때 매우 유용

chat=ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1
)

chat.invoke("Give me a recipe of 김치볶음밥")

AIMessage(content='Ingredients:\n- 2 cups cooked rice\n- 1 cup kimchi, chopped\n- 1/2 cup kimchi juice\n- 1/2 cup diced pork belly or bacon\n- 1/2 onion, diced\n- 2 cloves garlic, minced\n- 1 tbsp gochujang (Korean red chili paste)\n- 1 tbsp soy sauce\n- 1 tbsp sesame oil\n- 1 green onion, chopped\n- 1 fried egg (optional)\n- Sesame seeds for garnish\n\nInstructions:\n1. Heat a large skillet over medium heat and add the diced pork belly or bacon. Cook until browned and crispy.\n2. Add the diced onion and minced garlic to the skillet and cook until softened.\n3. Add the chopped kimchi and cook for a few minutes until heated through.\n4. Stir in the gochujang, soy sauce, and kimchi juice. Mix well.\n5. Add the cooked rice to the skillet and stir to combine with the kimchi mixture.\n6. Drizzle with sesame oil and continue to cook for a few more minutes, stirring occasionally.\n7. Remove from heat and garnish with chopped green onion and sesame seeds.\n8. Serve hot with a fried egg on top,

In [55]:
chat.invoke("Give me a recipe of 김치볶음밥") #똑같은 질문을 두 번째 함 -> 답변하는데 0초 소요. 메모리에서 들고 왔으니까.

AIMessage(content='Ingredients:\n- 2 cups cooked rice\n- 1 cup kimchi, chopped\n- 1/2 cup kimchi juice\n- 1/2 cup diced pork belly or bacon\n- 1/2 onion, diced\n- 2 cloves garlic, minced\n- 1 tbsp gochujang (Korean red chili paste)\n- 1 tbsp soy sauce\n- 1 tbsp sesame oil\n- 1 green onion, chopped\n- 1 fried egg (optional)\n- Sesame seeds for garnish\n\nInstructions:\n1. Heat a large skillet over medium heat and add the diced pork belly or bacon. Cook until browned and crispy.\n2. Add the diced onion and minced garlic to the skillet and cook until softened.\n3. Add the chopped kimchi and cook for a few minutes until heated through.\n4. Stir in the gochujang, soy sauce, and kimchi juice. Mix well.\n5. Add the cooked rice to the skillet and stir to combine with the kimchi mixture.\n6. Drizzle with sesame oil and continue to cook for a few more minutes, stirring occasionally.\n7. Remove from heat and garnish with chopped green onion and sesame seeds.\n8. Serve hot with a fried egg on top,

### 4.6 Serialization
* 모델을 저장 & 불러오기

In [56]:
# 한 번 API 콜을 할 떄 소요되는 비용 체크
from langchain_openai import ChatOpenAI # type: ignore
from langchain.callbacks import get_openai_callback # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.1,
)

with get_openai_callback() as usage:
    kimchi_recipe = chat.invoke("Tell me how I can make Kimchi(겉절이)")
    print(usage)
    print(usage.total_cost) #가격에 대해서만 알고 싶다면
    print(usage.prompt_tokens) #내가 모델에 건네준 토큰
    print(usage.completion_tokens) #모델이 생성한 토큰

print(kimchi_recipe)


Tokens Used: 430
	Prompt Tokens: 22
		Prompt Tokens Cached: 0
	Completion Tokens: 408
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0006230000000000001
content='Kimchi is a traditional Korean side dish made of fermented vegetables, most commonly napa cabbage and Korean radish. Here is a simple recipe to make kimchi at home:\n\nIngredients:\n- 1 napa cabbage\n- 1 Korean radish\n- 1/2 cup of coarse sea salt\n- 1/4 cup of Korean red pepper flakes (gochugaru)\n- 1 tablespoon of minced garlic\n- 1 tablespoon of minced ginger\n- 1 tablespoon of fish sauce\n- 1 tablespoon of sugar\n- 2 green onions, chopped\n- 1 carrot, julienned\n\nInstructions:\n1. Cut the napa cabbage in half lengthwise, then cut each half into quarters. Remove the core and chop the cabbage into bite-sized pieces. Peel the Korean radish and cut it into thin matchsticks.\n\n2. In a large bowl, dissolve the sea salt in water. Add the cabbage and radish, making sure they are fully submerged in the brine. L

In [65]:
#모델 저장하기
from langchain_openai import ChatOpenAI # type: ignore
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_completion_tokens=500, #말 많은거 싫어
    temperature=0.1 #너무 창의적인거 싫어
)
# chat.save("model.json") #model.json 파일이 생성됨 -> 더 이상 지원하지 않음. joblib 혹은 pickle을 사용해야 함.

# #모델 불러오기
from langchain.llms.loading import load_llm
# chat = load_llm("model.json")

## 5. MEMORY
* OpenAI의 Chat은 기본적으로 memory를 제공하지 않음 (stateless)
* 랭체인을 이용해서 implement

### 5.0 ConversationBufferMemory
* <b>Conversation Buffer Memory</b>: 대화 전체를 저장. prompt가 길어짐 -> 비싸!

In [67]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True) #return_messages=False를 주면 메시지 객체가 아니라 문자열을 반환

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

### 5.1 ConversationBufferWindowMemory
* <b>Conversation Buffer Window Memory</b>: 최근 n개의 대화를 저장 -> 최근 대화만 기억할 것

In [1]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message("1","1") #이 부분은 버려짐
add_message("2","2")
add_message("3","3")
add_message("4","4")
add_message("5","5")

memory.load_memory_variables({})



/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_88588/1223011226.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

### 5.2 ConversationSummaryMemory
* <b>Conversation Summary Memory</b>: llm을 이용. 대화가 길어질수록 유용해짐.

In [74]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi! I'm Boyon, I live in Paris.", "Wow that is so cool!")
add_message("Paris is very beautiful.", "I wish I could go")
get_history() #진행된 대화가 별로 없음 -> 초반엔 대화보다 summary가 더 긺.

{'history': 'The human introduces themselves as Boyon from Paris. The AI responds with enthusiasm at the information shared, expressing a desire to visit the beautiful city of Paris.'}

### 5.3 ConversationSummaryBufferMemory
* <b>Conversation Summary Buffer Memory</b>: max_token_limit에 달하면, 최근 메시지를 제외한 메시지들을 summarize

In [75]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=20, #최근 max_token_limit개 토큰의 interactions를 제외하고 summarize
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi! I'm Boyon, I live in Paris.", "Wow that is so cool!")
add_message("Paris is very beautiful.", "I wish I could go")
get_history() #진행된 대화가 별로 없음 -> 초반엔 대화보다 summary가 더 긺.

/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/1068426509.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


{'history': [SystemMessage(content='Boyon introduces themselves and mentions they live in Paris. The AI expresses admiration for this fact. Boyon agrees, stating that Paris is very beautiful.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go', additional_kwargs={}, response_metadata={})]}

### 5.4 ConversationKGMemory
* <b>Conversational Knowledge Graph Memory</b>: 대화 중인 entity의 knowledge graph를 만듦.
* 대화에서 추출한 지식(triple: 주어-동사-목적어) 를 그래프 형태로 저장.
* 예: "Alice lives in Paris" → (Alice, lives in, Paris)
* 지식 기반 챗봇이나 엔티티 추론 등에 유용.
* 일반 대화보다는 정보 추출이 중요한 상황에서 사용.
* <code>.save_context()</code>: 입력/출력 내용을 분석해 triple로 바꾸기 위해 LLM을 호출
* <code>.load_memory_variables</code>: keyword 기반으로 context를 반환. 실제 응답은 하지 않고, 문맥을 제공. API 호출 x




In [86]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)
memory=ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message("Hello! I'm Boyoon.", "Hi Boyoon! How can I help you today?")
add_message("What's your name?", "I'm an AI assistant created to help you.")
add_message("I like pizza.", "Pizza is delicious! What's your favorite topping?")
print(memory.load_memory_variables({"input":"Who is Boyoon?"})) 
#entity의 knowledge graph를 만들어 요약본을 생성, 메모리에 저장

{'history': [SystemMessage(content='On Boyon: Boyon lives in Paris.', additional_kwargs={}, response_metadata={})]}


### 5.5 Memory on LLMChain
* LLM chain: off-the-shelf chain. 일반적인 목적. 빠르게 시작할 수 있음.

In [2]:
# LLMChain에 우리가 가진 메모리 넣기
from langchain.chains import LLMChain

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "chat_history" #template안에 넣을 history의 위치를 알려줌
)

template = """
    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, #chain의 프롬프트 로그들을 확인할 수 있음
)

print(chain.predict(question="My name is Boyoon."))
print(chain.predict(question="I live in Paris since 7 years ago, but I was born and raised in Seoul."))
print(chain.predict(question="While I was traveling, a stranger asked me where I'm from. How do I answer to that?"))

/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_88588/2986166140.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_88588/2986166140.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
Python(14759) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.




> Entering new LLMChain chain...
Prompt after formatting:

    
    Human: My name is Boyoon.
    You: 


> Finished chain.
Nice to meet you, Boyoon! How can I assist you today?


> Entering new LLMChain chain...
Prompt after formatting:

    Human: My name is Boyoon.
AI: Nice to meet you, Boyoon! How can I assist you today?
    Human: I live in Paris since 7 years ago, but I was born and raised in Seoul.
    You: 


> Finished chain.
That's interesting! How do you like living in Paris compared to Seoul?


> Entering new LLMChain chain...
Prompt after formatting:

    System: The human introduces themselves as Boyoon. The AI responds by greeting Boyoon and asking how it can assist them today.
Human: I live in Paris since 7 years ago, but I was born and raised in Seoul.
AI: That's interesting! How do you like living in Paris compared to Seoul?
    Human: While I was traveling, a stranger asked me where I'm from. How do I answer to that?
    You: 


> Finished chain.
You can simply ans

### 5.6 Chat Based Memory
* memory.load_memory_variables({})가 문자열이 아니라 메시지 객체를 반환하게 합시다

In [3]:
# LLMChain에 우리가 가진 메모리 넣기
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "chat_history",
    return_messages=True
)

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"), #누가 보냈는지 알 수 없고 예측하기 어려운 양의 메시지를 넣을 수 있음
    ("human","{question}"),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

print(chain.predict(question="My name is Boyoon."))
print(chain.predict(question="I live in Paris since 7 years ago, but I was born and raised in Seoul."))
print(chain.predict(question="What's my name?"))



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Boyoon.

> Finished chain.
Hello Boyoon! How can I assist you today?


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Boyoon.
AI: Hello Boyoon! How can I assist you today?
Human: I live in Paris since 7 years ago, but I was born and raised in Seoul.

> Finished chain.
That's interesting! It sounds like you have a multicultural background. How are you finding life in Paris compared to Seoul?


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
System: The human introduces themselves as Boyoon and mentions living in Paris for the past 7 years, but being born and raised in Seoul. The AI greets Boyoon and asks how it can assist.
AI: That's interesting! It sounds like you have a multicultural background. How are you finding life in Pa

### 5.7 LCEL Based Memory

In [104]:
from langchain.schema.runnable import RunnablePassthrough

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "history",
    return_messages=True
)

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"), #누가 보냈는지 알 수 없고 예측하기 어려운 양의 메시지를 넣을 수 있음
    ("human","{question}")
])

def load_memory(_):
    print(_)
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context(
        {"input":question}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("My name is Boyoon")
invoke_chain("What is my name?")

input_variables=['history', 'question'] input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.func

## 6 RAG
* RAG: Retrieval Augmented Generation. 모델이 잘 모르는 정보를 제공하고, 그를 기반으로 우리 질문에 대답하게 함.

### 6.1 Data Loaders and Splitters
* <b>Retrieval</b>: langchain의 모듈
* <b>embedding</b>: 텍스트를 숫자로 변환
* <b>loader</b>: 소스에서 데이터를 추출라고 랭체인에게 가져다 주는 코드
* <b>document</b>: 텍스트 덩어리. 너무 크면 splitter로 분리 -> 질문에 답변하는 데 필요한 부분만 전달

In [106]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredFileLoader #어떤 확장자의 파일이든 불러올 수 있음
from langchain.text_splitter import RecursiveCharacterTextSplitter #generic. 아주 일반적으로 사용하는 splitter. 문장이나 문단 끝마다 split

llm = ChatOpenAI(temperature=0.1)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, #chunk_size: 더 작은 단위의 document로 잘라냼. but 문장도 다 잘라버림
    chunk_overlap=50 #문장을 자르지 않기 위해 앞의 토큰 몇 개를 뒤에 붙임
) 
loader = TextLoader("./files/chapter_one.txt")

# loader.load() #전부 load. 하나의 document가 들어있는 리스트가 반환
loader.load_and_split(text_splitter=splitter) #chunk_size를 주지 않으면 11개의 document가 들어있는 리스트가 반환


[Document(metadata={'source': './files/chapter_one.txt'}, page_content='Part One'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='1'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a'),
 Document(metadata={'source': './files/chapter_one.txt'}, p

In [108]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter #특정 separator를 기준으로 split

llm = ChatOpenAI(temperature=0.1)
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600, #separator 유무에 따라 더 커질수도, 작아질 수도 있음. chunk의 사이즈를 계산하는 데에는 파이썬 표준 라이브러리의 len 함수 사용
    chunk_overlap=100
)
loader = TextLoader("./files/chapter_one.txt")

loader.load_and_split(text_splitter=splitter) #11개의 document가 들어있는 리스트가 반환됨



Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

[Document(metadata={'source': './files/chapter_one.txt'}, page_content='Part One\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in pre

### 6.2 Tiktoken
* chunk의 사이즈를 계산하는 데에는 파이썬 표준 라이브러리의 len 함수 사용. length_funtion arg를 사용해서 token을 세는 것으로 변경 가능
* openai의 tokenizer(tiktoken) : https://platform.openai.com/tokenizer
* The models learn to understand the statistical relationships between these tokens, and excel at producing the next token in a sequence of tokens.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

llm = ChatOpenAI(temperature=0.1)
splitter = CharacterTextSplitter.from_tiktoken_encoder( #이제 우리가 size를 계산하는 법이 OpenAI가 size를 계산하는 법과 같음
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./files/chapter_one.txt")

loader.load_and_split(text_splitter=splitter)



### 6.3 Vectors
* vector: 단어를 숫자로 변환. 단어 사이의 연산이 가능해짐
* https://turbomaze.github.io/word2vecjson/ : 비슷한 벡터를 가진 단어들을 볼 수 있음

### 6.4 Vector Store
* Chroma: 오픈소스 vector store. 클라우드가 아니라 우리 컴퓨터에서 실행됨. 우린 나중에 pinecone과 같은 클라우드 벡터 스토어를 쓸거임.

In [110]:
from langchain_openai import OpenAIEmbeddings #OpenAI가 개발, 랭체인이 지원

embedder = OpenAIEmbeddings()
hi_vector = embedder.embed_query("Hi") #단어 하나를 embed
print(len(hi_vector)) #1536 dimension

vector = embedder.embed_documents([ #document를 embed. 우리는 document를 임베딩할거임
    "소리에 놀라지 않는 사자와 같이",
    "그물에 걸리지 않는 바람과 같이",
    "흙탕물에 더럽혀지지 않는 연꽃과 같이",
    "무소의 뿔처럼 혼자서 가라"
])
print(len(vector)) #4개
print(len(vector[0])) #각각의 벡터는 1536 dimension을 가짐

1536
4
1536


In [126]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings #model: str = "text-embedding-ada-002", 저렴함
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/") #여기에 캐시가 생성될거임

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs=loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings) #vector 생성. 이 라인을 실행하면 요금을 지불해야 함 / embedding을 캐시한 이후로는 캐시된 embedding을 불러올거임

results = vectorstore.similarity_search("Where does Winston live?") #결과로 document 4개를 반환함. query와 비슷한 문서. 이 문서랑 내 질문을 같이 모델에 전달해주면 대답해줄거임.
print(results)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


[Document(metadata={'source': './files/chapter_one.txt'}, page_content='Part One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was

### 6.5 Langsmith
* 우리의 체인이 뭘 하고 있는지 시각적으로 볼 수 있음
* https://smith.langchain.com/
* .env파일 안에 API_KEY 넣어

### 6.6 RetrievalQA
* 많은 off-the-shelf 체인이 있음. 그러나 현재 권장되는 방식은 LCEL로 내 체인을 만드는 것
* <b>RetrievalQA</b>: 랭체인에서 RAG 작업을 쉽게 해주는 고수준 래퍼 클래스. 
    - retriever: 관련 문서를 가져옴
    - chain_type: 문서들을 어떤 방식으로 LLM에 전달할지 결정
    - llm: 언어모델
* chain types
    - <b>stuff</b>: 모든 문서를 한 번에 LLM에 넣고 한 번에 답변
    - <b>map_reduce</b>: 각 문서에서 부분 답변 생성(map) -> 종합해서 최종 답변 생성(reduce)
    - <b>refine</b>: 첫 문서로 초안 작성 -> 이후 문서로 점점 개선(refine)
    - <b>map_rerank</b>: 각 문서에서 답변 생성 -> 가장 좋은 답 하나를 선택. 부분 정보가 무시될 수 있음


In [152]:
#off-the-shelf체인
from langchain.chains import RetrievalQA

#랭스미스와의 연동을 위해 필요 (2024년 말 업데이트 이후 자동 트레이싱이 비활성화 됨. 수동으로 callbacks를 줘야 함)
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma # Chroma랑 FAISS는 비슷. 니코쌤 경험으로는 FAISS가 나을 때가 있다고 함. vectorstore를 바꾸려면 cache도 삭제해야 함.

#langsmith에서 tracing이 제대로 되도록
client = Client()
tracer = LangChainTracer(client=client)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[tracer])

#document를 load후 split
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs=loader.load_and_split(text_splitter=splitter)

#embedding(&caching)을 통해 vectorstore로 변환
cache_dir = LocalFileStore("./.cache/") #여기에 캐시가 생성될거임
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = Chroma.from_documents(docs, cached_embeddings) #vector 생성. 이 라인을 실행하면 요금을 지불해야 함 / embedding을 캐시한 이후로는 캐시된 embedding을 불러올거임

#vectorstore 테스트
results = vectorstore.similarity_search("Where does Winston live?") #결과로 document 4개를 반환. 질문을 임베딩해야 하기 때문에 API 호출이 1회 일어남.

#RetrievalQA 테스트 (off-the-shelf chains). LangSmith를 통해 체인 내부에서 어떤 일이 일어나는지 확인 가능
chain = RetrievalQA.from_chain_type( #일종의 constructor
    llm=llm,
    chain_type="map_rerank", #default: stuff.
    retriever=vectorstore.as_retriever() #인터페이스. 우리 vector store말고 다른 곳에서도 document를 가져올 수 있음. store할 수는 없음. general. 이 라인은 우리 벡터 스토어를 가져와서 retriever로 변환함.
)

chain.run("Where does Winston live? And describe this place.")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  """Call apply and then parse the results."""


'Winston lives in Victory Mansions, a building with a hallway that smells of boiled cabbage and old rag mats. The flat is seven flights up, and the building has a poster with an enormous face that says "BIG BROTHER IS WATCHING YOU." Inside the flat, there is a telescreen that cannot be shut off completely.'

### 6.8 Stuff LCEL Chain
* RetrievalQA를 사용하지 않고 stuff chain을 구현

In [156]:
#
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough #input이 다음 체인으로 넘어가게 해줌

#랭스미스와의 연동을 위해 필요 (2024년 말 업데이트 이후 자동 트레이싱이 비활성화 됨. 수동으로 callbacks를 줘야 함)
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain_openai import ChatOpenAI

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma

client = Client()
tracer = LangChainTracer(client=client)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[tracer])

#load and split
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs=loader.load_and_split(text_splitter=splitter)

#embed, cache and create vectorstore
cache_dir = LocalFileStore("./.cache/") 
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = Chroma.from_documents(docs, cached_embeddings)

#test vectorstore
results = vectorstore.similarity_search("Where does Winston live?")

#create stuff documents chain
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}"),
    ("human","{question}")
])

chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

chain.invoke("Where does Winston live? And describe this place.")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)\')))"))\nContent-Length: None\nAPI Key: lsv2_********************************************d3')


AIMessage(content='Winston lives in Victory Mansions, which is a dilapidated apartment building with a gritty and unpleasant hallway that smells of boiled cabbage and old rag mats. The building has a faulty lift, forcing Winston to climb seven flights of stairs to reach his flat. Inside his flat, there is a telescreen that cannot be completely turned off, constantly broadcasting propaganda messages.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 1944, 'total_tokens': 2019, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSPZHuDPCdrd04uG34ONPY1IjBFZN', 'finish_reason': 'stop', 'logprobs': None}, id='run-bea9d571-c328-4ba2-adff-bd5d9dbeb991-0', usage_metadata={'input_tokens': 1944, 'out

Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))"))
Content-Length: 10511
API Key: lsv2_********************************************d3
post: trace=bea9d571-c328-4ba2-adff-bd5d9dbeb991,id=bea9d571-c328-4ba2-adff-bd5d9dbeb991
Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: 

### 6.9 Map Reduce LCEL Chain
* <b>map_reduce</b>: 각 문서에서 부분 답변 생성(map) -> 종합해서 최종 답변 생성(reduce)
* chain 3개 만들거임
    - map_doc_chain: 각각의 문서에 대해 부분 답변 생성
    - map_chain: retriever가 가져다준 문서들을 map_doc_chain에 넣고 결과물을 하나의 text로 만듦
    - chain: map_chain이 만들어준 text를 바탕으로 최종 답변 생성

In [159]:

from langchain.schema.runnable import RunnableLambda #chain과 그 내부 어디에서든 function을 호출할 수 있게 해줌

#랭스미스와의 연동을 위해 필요 (2024년 말 업데이트 이후 자동 트레이싱이 비활성화 됨. 수동으로 callbacks를 줘야 함)
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma

client = Client()
tracer = LangChainTracer(client=client)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[tracer])

# load and split
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs=loader.load_and_split(text_splitter=splitter)

#embed, cache and create vectorstore
cache_dir = LocalFileStore("./.cache/") 
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = Chroma.from_documents(docs, cached_embeddings)

#각 문서에서 부분 답변을 생성하는 체인
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", "Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.\n------\n{portion}"),
    ("human","{question}")
])
map_doc_chain = map_doc_prompt | llm

# def map_docs(inputs):
#     documents = inputs['documents']
#     question = inputs['question']
#     results = []
#     for document in documents:
#         result = map_doc_chain.invoke({
#             "portion":document.page_content,
#             "question":question
#         }).content
#         results.append(result)
#     res = "\n\n".join(results)
#     return res

#위 함수를 개선
#모든 doc에 대해 map_doc_chain을 invoke -> 결과를 하나의 문자열로 만들어서 반환
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({"portion":doc.page_content, "question":question}).content for doc in documents)

#retriever가 가져온 문서들을 map_docs에 넣어 -> 각각의 문서에 대한 답변들을 합해서 하나의 text를 만듦
retriever = vectorstore.as_retriever()
map_chain = {"documents":retriever, "question":RunnablePassthrough()} | RunnableLambda(map_docs) #retriever가 자동으로 question을 입력값으로 받음

#마지막 체인. "아주 긴 text를 바탕으로 답변을 생성하라"
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say that you don't know. Don't try to make up an answer.\n-----\n{context}"),
    ("human", "{question}")
])
chain = {"context":map_chain, "question":RunnablePassthrough()} | final_prompt| llm
chain.invoke("Where does Winston live? And describe this place.")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


AIMessage(content="Winston lives in Victory Mansions, a poorly maintained building with a hallway that smells of boiled cabbage and old rag mats. The flat is on the seventh floor, and the building has a poster with an enormous face of a man about forty-five years old with a black mustache and ruggedly handsome features. The building's lift seldom works, the electric current is cut off during daylight hours as part of an economy drive in preparation for Hate Week, and it is equipped with a telescreen that constantly monitors the residents.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 521, 'total_tokens': 627, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSQI6jPFDzXeU7bNgKHfLkR7nW

## 7 DOCUMENT GPT

### 7.0 Introduction
* Streamlit이 작동하는지 확인 - Home.py를 만들고 아래 코드를 run
* 새로고침을 하지 않아도 변경사항이 반영되게 할 수 있음

In [ ]:
import streamlit as st
st.header("Hello world")
st.subheader("Welcome to Strealit")

### 7.1 Magic
* st.write(): 매직. 스위스 군용칼 같음. 안에 아무거나 다 넣을 수 있음. 멋있게 출력해줌.
* 사실 함수 안에 안넣어도 display됨

### 7.2 Data Flow
* <code>st.selectbox()</code>에서 뭔가 선택하면 페이지가 refresh됨 (전체 파일이 다시 실행됨)
* ReactJS나 Flutter와는 다름 (새로운 부분만 refresh되는게 아님)
* <code>name = st.text_input("What is your name?")</code>: text input을 받음. Enter를 쳐야 페이지가 refresh됨
* Streamlit에는 cache를 제공하는 매커니즘이 있어서 어떤 것은 다시 실행되지 않음.

In [ ]:
import streamlit as st
from datetime import datetime

today=datetime.today().strftime("%H:%M:%S")

st.title(today)

model = st.selectbox("Choose your model", ("GPT-3", "GPT-4"))

if model=="GPT-3":
    st.write("cheap")
else:
    st.write("not cheap")

name = st.text_input("What is your name?")
st.write(name)

value = st.slider("temperature", min_value=0.1, max_value=1.0)
st.write(value)

### 7.3 Multi Page
* Streamlit에서 sidebar를 만들 때 사용하는 패턴이 있음
    1. <code>st.sidebar.</code> 뒤에 title, text_input 등을 쓰면 됨
    2. <code>with st.sidebar:</code> 내부에 코드 작성. <code>.sidebar</code>를 반복하지 않아도 됨
* pages를 추가: pages라는 이름의 폴더를 추가하면 됨. streamlit은 pages라는 이름의 폴더를 찾음

In [ ]:
import streamlit as st

st.set_page_config(
    page_title="FullstackGPT Home", 
    page_icon="🤖") #탭에 표시될 page_title을 설정

st.title("title")

#option1
# st.sidebar.title("sidebar title")
# st.sidebar.text_input()

#option2
with st.sidebar:
    st.title("sidebar title")
    st.text_input("write your name")

tab_one, tab_two, tab_three = st.tabs(["A","B","C"])

with tab_one:
    st.write("a")

with tab_two:
    st.write("b")

with tab_three:
    st.write("c")

### 7.4 Chat Messages
* <b>session state</b>: streamlit의 기능. 여러번 재실행해도 데이터가 보존됨.

In [4]:
# 유저가 메시지를 보내면 streamlit은 그 메시지를 반복. 챗봇처럼 보이게 함.
import streamlit as st
import time

st.title("Documnet GPT")

# with st.chat_message("human"):
#     st.write("Hello!")

# with st.chat_message("ai"): #chat_message의 파라미터 name : "user", "assistant", "ai", "human", or str
#     st.write("how are you?")

# with st.status("Embedding file....", expanded=True) as status:
#     time.sleep(2)
#     st.write("Getting the file")
#     time.sleep(2)
#     st.write("Embedding the file")
#     time.sleep(2)
#     st.write("Caching the file")
#     status.update(label="Error", state="error")

# st.chat_input("Send a message to the AI")

if "messages" not in st.session_state:
    st.session_state["messages"] = [] #session state를 이용하면 whole code가 다시 실행되더라도 이 안의 데이터는 살아있음(cached). 단 새로고침을 하면 사라짐.

def send_message(message, role, save=True):
    with st.chat_message(role):
        st.write(message)
    if save:
        st.session_state["messages"].append({"message":message, "role":role})

for message in st.session_state["messages"]:
    send_message(message["message"], message["role"], save=False)

message = st.chat_input("Send a message to the AI")

if message:
    send_message(message, "human") #우리가 메시지를 보낼 때마다 대화의 모든 messages를 다시 그림 (캐시로부터)
    time.sleep(1)
    send_message(f"You said {message}", "ai")

    with st.sidebar:
        st.write(st.session_state)

2025-05-08 11:57:18.390 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 11:57:18.646 
  command:

    streamlit run /Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-08 11:57:18.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 11:57:18.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 11:57:18.647 Session state does not function when running a script without `streamlit run`
2025-05-08 11:57:18.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 11:57:18.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 11:57:18.648 Thread

### 7.6 Uploading Documents
* DocumentGPT.py가 환경변수를 제대로 못읽어올 수도 있음. -> .streamlit 디렉토리, 그 안에 secrets.toml 파일을 만들고 API key들을 복사 붙여넣기하면 해결

In [ ]:
import streamlit as st
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
# from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def embed_file(file):
    st.write(file)
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    # st.write(file_content, file_path)
    with open(file_path, "wb") as f:
        f.write(file_content)
        
    #여기서부턴 지난 수업내용 copy-paste
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[LangChainTracer(client=Client())])

    # load and split
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100
    )
    loader = UnstructuredFileLoader(file_path)
    docs=loader.load_and_split(text_splitter=splitter)

    #embed, cache, and create vectorstore
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}") 
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        embeddings, cache_dir
    )
    vectorstore = Chroma.from_documents(docs, cached_embeddings)

    #각 문서에서 부분 답변을 생성하는 체인
    map_doc_prompt = ChatPromptTemplate.from_messages([
        ("system", "Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.\n------\n{portion}"),
        ("human","{question}")
    ])
    map_doc_chain = map_doc_prompt | llm

    #모든 doc에 대해 map_doc_chain을 invoke -> 결과를 하나의 문자열로 만들어서 반환
    def map_docs(inputs):
        documents = inputs["documents"]
        question = inputs["question"]
        return "\n\n".join(map_doc_chain.invoke({"portion":doc.page_content, "question":question}).content for doc in documents)

    #retriever가 가져온 문서들을 map_docs에 넣어 -> 각각의 문서에 대한 답변들을 합해서 하나의 text를 만듦
    retriever = vectorstore.as_retriever()
    return retriever


st.title("Documnet GPT")

st.markdown("""
Welcome!
            
Use this chatbot to ask questions to an AI about your files!            
""")

file = st.file_uploader("Upload a .txt .pdf or .docx file", type=["pdf", "txt", "docx"])

if file:
    retriever = embed_file(file) #사용자가 뭔가 입력할 때마다 전체 함수가 실행됨 -> embeddings가 cache가 돼있어도 시간이 걸림.
    s = retriever.invoke("winston")
    st.write(s)

### 7.7 Chat History
* <code>@st.cache_data</code>: 데코레이터. cache functions that return data. 이미 캐시된 리턴값이 있으면 다시 실행하지 않음.
* 위 데코레이터를 쓰고 에러 발생하면 <code>@st.cache_resource(show_spinner="Embedding file...")</code>를 쓰면 됨.

In [ ]:
import streamlit as st
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
# from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

@st.cache_resource(show_spinner="Embedding file...")
def embed_file(file):
    st.write(file)
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    # st.write(file_content, file_path)
    with open(file_path, "wb") as f:
        f.write(file_content)
        
    #여기서부턴 지난 수업내용 copy-paste
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[LangChainTracer(client=Client())])

    # load and split
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100
    )
    loader = UnstructuredFileLoader(file_path)
    docs=loader.load_and_split(text_splitter=splitter)

    #embed, cache, and create vectorstore
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}") 
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        embeddings, cache_dir
    )
    vectorstore = Chroma.from_documents(docs, cached_embeddings)

    #retriever가 가져온 문서들을 map_docs에 넣어 -> 각각의 문서에 대한 답변들을 합해서 하나의 text를 만듦
    retriever = vectorstore.as_retriever()
    return retriever

def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        st.session_state["messages"].append({"message":message, "role":role})

st.set_page_config(page_title="DocumentGPT", page_icon="📄")

def paint_history():
    for message in st.session_state["messages"]:
        send_message(message["message"], message["role"], save=False)

st.title("Documnet GPT")

st.markdown("Welcome!\n\nUse this chatbot to ask questions to an AI about your files!\n\nUpload your files in the sidebar")

with st.sidebar:
    file = st.file_uploader("Upload a .txt .pdf or .docx file", type=["pdf", "txt", "docx"])

if file:
    retriever = embed_file(file) #사용자가 뭔가 입력할 때마다 전체 함수가 실행됨 -> embeddings가 cache가 돼있어도 시간이 걸림.

    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()

    message = st.chat_input("Ask questions about your file...")
    if message:
        send_message(message, "human")
else:
    st.session_state["messages"] = [] #파일이 없어지면(유저가 x 클릭) messages를 초기화

### 7.8 Chain

In [ ]:
import streamlit as st
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


@st.cache_resource(show_spinner="Embedding file...")
def embed_file(file):
    st.write(file)
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    # st.write(file_content, file_path)
    with open(file_path, "wb") as f:
        f.write(file_content)

    # load and split
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100
    )
    loader = UnstructuredFileLoader(file_path)
    docs=loader.load_and_split(text_splitter=splitter)

    #embed, cache, and create vectorstore
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}") 
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        embeddings, cache_dir
    )
    vectorstore = Chroma.from_documents(docs, cached_embeddings)

    #retriever가 가져온 문서들을 map_docs에 넣어 -> 각각의 문서에 대한 답변들을 합해서 하나의 text를 만듦
    retriever = vectorstore.as_retriever()
    return retriever

def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        st.session_state["messages"].append({"message":message, "role":role})

st.set_page_config(page_title="DocumentGPT", page_icon="📄")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, callbacks=[LangChainTracer(client=Client())])

def paint_history():
    for message in st.session_state["messages"]:
        send_message(message["message"], message["role"], save=False)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.\n\nContext: {context}"),
    ("human", "{question}")
])

st.title("Documnet GPT")

st.markdown("Welcome!\n\nUse this chatbot to ask questions to an AI about your files!\n\nUpload your files in the sidebar")

with st.sidebar:
    file = st.file_uploader("Upload a .txt .pdf or .docx file", type=["pdf", "txt", "docx"])

if file:
    retriever = embed_file(file) #사용자가 뭔가 입력할 때마다 전체 함수가 실행됨 -> embeddings가 cache가 돼있어도 시간이 걸림.

    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()

    message = st.chat_input("Ask questions about your file...")
    if message:
        send_message(message, "human")
        chain = {"context":retriever | RunnableLambda(format_docs), "question":RunnablePassthrough()} | prompt | llm
        response = chain.invoke(message)
        send_message(response.content, "ai")

else:
    st.session_state["messages"] = [] #파일이 없어지면(유저가 x 클릭) messages를 초기화

### 7.9 Streaming
* callbackhandler: llm의 event를 listen하는 아주 멋진 class

In [ ]:
import streamlit as st
from langsmith import Client
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.callbacks.base import BaseCallbackHandler


@st.cache_resource(show_spinner="Embedding file...")
def embed_file(file):
    st.write(file)
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    # st.write(file_content, file_path)
    with open(file_path, "wb") as f:
        f.write(file_content)

    # load and split
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100
    )
    loader = UnstructuredFileLoader(file_path)
    docs=loader.load_and_split(text_splitter=splitter)

    #embed, cache, and create vectorstore
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}") 
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        embeddings, cache_dir
    )
    vectorstore = Chroma.from_documents(docs, cached_embeddings)

    #retriever가 가져온 문서들을 map_docs에 넣어 -> 각각의 문서에 대한 답변들을 합해서 하나의 text를 만듦
    retriever = vectorstore.as_retriever()
    return retriever

def save_message(message, role):
    st.session_state["messages"].append({"message":message, "role":role})

def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        save_message(message, role)

st.set_page_config(page_title="DocumentGPT", page_icon="📄")

class ChatCallbackHandler(BaseCallbackHandler):
    message = ""
    def on_llm_start(self, *args, **kwargs): # can take unlimited arguments(1,2,3,..) and keyword arguments (a=5)
        self.message_box = st.empty() #빈 위젯을 제공
    def on_llm_end(self, *args, **kwargs):
        save_message(self.message, "ai")
        # with st.sidebar:
        #     st.write("llm ended!")
    def on_llm_new_token(self, token:str, *args, **kargs): #token은 LLM이 실시간으로 보내는 메시지. 토큰이 도착할 때마다 message_box에 추가 (화면에 출력됨)
        self.message += token
        self.message_box.markdown(self.message)

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0.1, 
    callbacks=[LangChainTracer(client=Client()), ChatCallbackHandler()],
    streaming=True #ChatOpenAI는 지원. 다른 llm은 지원 안할 수도 있음.
)

def paint_history():
    for message in st.session_state["messages"]:
        send_message(message["message"], message["role"], save=False)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.\n\nContext: {context}"),
    ("human", "{question}")
])

st.title("Documnet GPT")

st.markdown("Welcome!\n\nUse this chatbot to ask questions to an AI about your files!\n\nUpload your files in the sidebar")

with st.sidebar:
    file = st.file_uploader("Upload a .txt .pdf or .docx file", type=["pdf", "txt", "docx"])

if file:
    retriever = embed_file(file) #사용자가 뭔가 입력할 때마다 전체 함수가 실행됨 -> embeddings가 cache가 돼있어도 시간이 걸림.

    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()

    message = st.chat_input("Ask questions about your file...")
    if message:
        send_message(message, "human")
        chain = {"context":retriever | RunnableLambda(format_docs), "question":RunnablePassthrough()} | prompt | llm
        with st.chat_message("ai"):
            response = chain.invoke(message)

else:
    st.session_state["messages"] = [] #파일이 없어지면(유저가 x 클릭) messages를 초기화

## 8 PRIVATE GPT

### 8.1 HuggingFaceHub
* Mistral을 사용할거임
* HuggingFace 모델을 실행시키는 법
  1. Hugging Face Inference API : 유료. HuggingFace 서버를 이용
  2. 우리 컴퓨터에 다운받고 실행 : LangChain의 HuggingFace adapter 사용 - token 필요. .env 파일에 넣을 것.
* 아래 코드는 Inference Providers를 사용. 무료 - 여러 모델을 테스트 하는 데에 유용함.

In [4]:
# HUGGINGFACEHUB_API_TOKEN이 정상적으로 로드되는지 체크
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드

# api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
# print(api_token)  # 정상적으로 출력되는지 확인


True

In [6]:
# parameters에 대한 doc은 아래에서 볼 수 있음
# https://huggingface.co/docs/inference-providers/tasks/text-generation
from langchain.llms import HuggingFaceHub, HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is {word}?")
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task='text-generation',
    # model_kwargs = {"max_new_tokens": 250,}, #생성될 텍스트. 이 부분을 주석처리하면 답변이 더 짧아짐.
    huggingfacehub_api_token=api_token
)
chain = prompt | llm
chain.invoke({"word":"HuggingFace"})


AttributeError: 'InferenceClient' object has no attribute 'post'

### 8.2 HuggingFacePipeline

In [14]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{word} is a ")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=-1 #0:GPU, -1:CPU / 0(GPU)는 맥북에서 안됨!
    # pipeline_kwargs={"max_new_tokens":50}

)

chain = prompt|llm
chain.invoke({"word":"LangChain"})


Device set to use cpu


"LangChain is a irc server that I use to communicate with other people. It was started by a friend of mine who was on the ground floor of our home in Texas for a while and he wanted to join up with us for a few weeks to watch a video of how fast they were going. I went to the link and found that it was a forum that was pretty similar to the ones we used to have in the past. I would connect with a few people, talk about stuff and then I would have a conversation with them. So for the first few weeks, we had a nice chat and then we were done. He and I had to split up and we were able to get more into each other's lives. We spent a lot of time together in Texas and our relationship was really good.\n\nSo the next time I'd go down to Texas to watch a movie I would sit back and watch the movie because I thought it was going to be fun. I think the movie was about a couple of guys who were doing an important thing and got caught up in the action and found out that they might be able to do som

### 8.3 GPT4All
* https://www.nomic.ai/gpt4all
* 우리 컴퓨터에서 로컬 모델을 실행하는 또 다른 방법
* 많이 바뀌었음. 수업처럼 모델을 직접 다운받지 못함. 일단은 그냥 넘어가는게 좋을듯.

In [6]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{word} is a ")

llm = GPT4All(model="gpt4all-falcon-q4_0.gguf")

chain = prompt|llm
chain.invoke({"word":"LangChain"})


FileNotFoundError: Model file does not exist: PosixPath('/Users/boyoon/.cache/gpt4all/gpt4all-falcon-q4_0.gguf')

### 8.4 Ollama
* https://ollama.com/
* 설치: <code>brew install ollama</code>
* 실행: <code>ollama serve</code> -> 이 터미널은 ollama 서버 프로세스 전용으로 잡힘
* 모델 실행: <code>ollama run mistral</code>
* mistral은 오픈소스고, Llama보다 더 잘 작동.

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-2025-08-07"
)

llm.invoke("Hi! Introduce yourself!")

AIMessage(content='Hi! I’m ChatGPT, your AI assistant. I can help you think, write, plan, code, learn, and solve problems—whether it’s drafting emails, explaining concepts, debugging, creating study plans, or brainstorming ideas.\n\nTips for best results:\n- Tell me your goal and audience.\n- Share any constraints (length, tone, deadline, tools).\n- Provide examples or a starting point if you have one.\n\nWhat would you like to work on today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 360, 'prompt_tokens': 12, 'total_tokens': 372, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C6Qk3FlLBUTBCXvbPL2aWPJq3yhEJ', 'finish_reason': 'stop', 'logprobs': None}, id='run--d6a680be-248e-4f0e-a157-ebc1e7f4d7f6-0',

#### 9.8 Function Calling
* 실제로 함수가 존재하지 않더라도 모델을 속여 output을 우리가 원하는 방식으로 제공하게 만들 수 있음.

In [20]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# def get_weather(lon, lat): #없어도 됨
#     print("calling an api...")
#     print(lon, lat)

function = { #schema. 실제로 존재하지 않는 함수의 schema를 줄 수도 있음.
    "name":"create_quiz",
    "description":"takes a list of questions and 4 answers per question, and returns a quiz",
    "parameters":{
        "type":"object", #LLM이 JSON 객체(dict)를 반환함
        "properties":{
            "questions":{ #questions 안에
                "type":"array", #배열이 있음
                "items":{ #각 배열의 아이템은
                    "type":"object", #또 다른 배열
                    "properties":{ #그 안에는
                        "question":{"type":"string"}, #question과
                        "answers":{ #answers가 있음
                            "type":"array",
                            "items":{
                                "type":"object",
                                "properties":{
                                    "answer":{"type":"string"},
                                    "correct":{"type":"boolean"}
                                },
                                "required":["answer","correct"]
                            }
                        }
                    }
                }
            }
        }
    }
}

llm = ChatOpenAI(temperature=0.1).bind(
    function_call={"name":"create_quiz"}, #우리가 준 함수를 반드시 사용하라고 강요할 수도 있음. 아니면 필요한 경우 사용하도록 내버려둘수도.
    functions=[function]) 
prompt = PromptTemplate.from_template("Make 10 quizes about {city}, with ")
chain = prompt | llm
response = chain.invoke({"city":"Paris"}).additional_kwargs["function_call"]["arguments"]
response



'{"questions":[{"question":"What is the capital of France?","answers":[{"answer":"Paris","correct":true},{"answer":"London","correct":false},{"answer":"Berlin","correct":false},{"answer":"Rome","correct":false}]},{"question":"Which famous museum in Paris is home to the Mona Lisa?","answers":[{"answer":"Louvre","correct":true},{"answer":"Musée d\'Orsay","correct":false},{"answer":"Centre Pompidou","correct":false},{"answer":"Musée Rodin","correct":false}]},{"question":"What is the iconic tower in Paris that offers panoramic views of the city?","answers":[{"answer":"Eiffel Tower","correct":true},{"answer":"Arc de Triomphe","correct":false},{"answer":"Montparnasse Tower","correct":false},{"answer":"Pantheon","correct":false}]},{"question":"Which river flows through Paris?","answers":[{"answer":"Seine","correct":true},{"answer":"Thames","correct":false},{"answer":"Danube","correct":false},{"answer":"Rhine","correct":false}]},{"question":"What is the famous avenue in Paris known for its lux

In [21]:
import json
json.loads(response)

{'questions': [{'question': 'What is the capital of France?',
   'answers': [{'answer': 'Paris', 'correct': True},
    {'answer': 'London', 'correct': False},
    {'answer': 'Berlin', 'correct': False},
    {'answer': 'Rome', 'correct': False}]},
  {'question': 'Which famous museum in Paris is home to the Mona Lisa?',
   'answers': [{'answer': 'Louvre', 'correct': True},
    {'answer': "Musée d'Orsay", 'correct': False},
    {'answer': 'Centre Pompidou', 'correct': False},
    {'answer': 'Musée Rodin', 'correct': False}]},
  {'question': 'What is the iconic tower in Paris that offers panoramic views of the city?',
   'answers': [{'answer': 'Eiffel Tower', 'correct': True},
    {'answer': 'Arc de Triomphe', 'correct': False},
    {'answer': 'Montparnasse Tower', 'correct': False},
    {'answer': 'Pantheon', 'correct': False}]},
  {'question': 'Which river flows through Paris?',
   'answers': [{'answer': 'Seine', 'correct': True},
    {'answer': 'Thames', 'correct': False},
    {'answer'

In [1]:
from transformers import pipeline

generator = pipeline("text-generation", model="sshleifer/tiny-gpt2")
prompt = "Hello, Hugging Face!"
output = generator(prompt, max_new_tokens=20)
print(output[0]['generated_text'])


/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, Hugging Face! Jr MotorolaSher vendors BrewScene subst scalp pawn circumcised AmphScene pawn Hancockdit conservation directly trilogy stairshibit
